<a href="https://colab.research.google.com/github/UKJaagadhep/Data-science-and-machine-learning/blob/main/Llama%202/Llama_2_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub # sice we get our Llama model from HuggingFace
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.7 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-70.1.1-py3-none-any.whl (883 kB)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 2.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 38.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 28.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 6.8 MB/s eta 0:00:00
    Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
    Creating /tmp/pip-build-env-gvtsvnlg/overlay/local/bin
    changing mode of /tmp/pip-build-env-gvtsvnlg/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-gvtsvnlg/overlay/local/bin/wheel to 755
    changing mode of /tmp/pip

In [2]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# **Downloading Model**

In [7]:
model_name = "TheBloke/Llama-2-7B-Chat-GGML"
model_basename = "llama-2-7b-chat.ggmlv3.q4_0.bin"

In [8]:
model_path = hf_hub_download(repo_id = model_name, filename = model_basename)

In [9]:
model_path

'/root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGML/snapshots/76cd63c351ae389e1d4b91cab2cf470aab11864b/llama-2-7b-chat.ggmlv3.q4_0.bin'

In [10]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path = model_path,
    n_threads = 2, # CPU cores
    n_batch = 512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers = 32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [11]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

# **Prompt and Response**

In [17]:
prompt = "Write program code for classification using random forest."
prompt_template = f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER : {prompt}

ASSISTANT :
'''

In [18]:
response = lcpp_llm(prompt = prompt_template,
                    max_tokens = 256,
                    temperature = 0.5,
                    top_p = 0.95,
                    repeat_penalty = 1.2,
                    top_k = 150,
                    echo = True)

Llama.generate: prefix-match hit


In [19]:
response

{'id': 'cmpl-9c952857-f734-4d7e-a754-0f22c90e7a23',
 'object': 'text_completion',
 'created': 1719712981,
 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGML/snapshots/76cd63c351ae389e1d4b91cab2cf470aab11864b/llama-2-7b-chat.ggmlv3.q4_0.bin',
 'choices': [{'text': "SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER : Write program code for classification using random forest.\n\nASSISTANT :\nClassification using Random Forest is a popular machine learning technique that involves training multiple decision trees on different subsets of the data and combining their predictions to make a final classification. Here's an example Python code for implementing Random Forest classifier in scikit-learn library:\n```python\nfrom sklearn import datasets\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.model_selection import train_test_split\n\n# Load the iris dataset\niris = datasets.load_iris()\nX = iris.data

In [20]:
print(response["choices"][0]["text"])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER : Write program code for classification using random forest.

ASSISTANT :
Classification using Random Forest is a popular machine learning technique that involves training multiple decision trees on different subsets of the data and combining their predictions to make a final classification. Here's an example Python code for implementing Random Forest classifier in scikit-learn library:
```python
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Load the iris dataset
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
y = iris.target

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Mak